In [ ]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

def preprocess(df, lags):
    df.columns = df.columns.str.strip()
    df['Date'] = pd.to_datetime(df['DATE'])
    df['Hour'] = df['TIME'].str.split(':').str[0].astype(int)
    df['Power'] = df['(kW)']
    df = df.drop(columns=['TIME', '(kW)'])
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    df['Month'] = df['Date'].dt.month
    df['DayOfYear'] = df['Date'].dt.dayofyear

    for lag in range(1, lags + 1):
        df[f'Lag{lag}'] = df['Power'].shift(lag)

    df = df.dropna()
    return df

def predict_next_year(file1, file2, file3, file4, file5, file6, file7):
    # Read the data for the years provided
    df_2016 = pd.read_csv(file1)
    df_2017 = pd.read_csv(file2)
    df_2018 = pd.read_csv(file3)
    df_2019 = pd.read_csv(file4)
    df_2020 = pd.read_csv(file5)
    df_2021 = pd.read_csv(file6)
    df_2022 = pd.read_csv(file7)

    # Define a range of potential lags and degrees to test
    potential_lags = range(1, 6)
    degrees = [1, 2, 3, 4, 5]

    best_lags_degrees_per_hour = {}
    test_mse_per_hour = []
    predictions_per_hour = {}

    for hour in range(24):
        best_lag = None
        best_degree = None
        best_val_mse = float('inf')

        for num_lags in potential_lags:
            # Preprocess data for the current number of lags
            df_train = preprocess(pd.concat([df_2016, df_2017, df_2018, df_2019, df_2020]), num_lags)
            df_val = preprocess(df_2021, num_lags)
            df_test = preprocess(df_2022, num_lags)

            hour_train_df = df_train[df_train['Hour'] == hour]
            hour_val_df = df_val[df_val['Hour'] == hour]
            hour_test_df = df_test[df_test['Hour'] == hour]

            if hour_train_df.empty or hour_val_df.empty or hour_test_df.empty:
                continue

            X_train = hour_train_df[['Hour', 'DayOfWeek', 'Month', 'DayOfYear'] + [f'Lag{lag}' for lag in range(1, num_lags + 1)]]
            y_train = hour_train_df['Power']

            X_val = hour_val_df[['Hour', 'DayOfWeek', 'Month', 'DayOfYear'] + [f'Lag{lag}' for lag in range(1, num_lags + 1)]]
            y_val = hour_val_df['Power']

            for degree in degrees:
                # Transform features to polynomial
                poly_features = PolynomialFeatures(degree=degree, include_bias=False)
                X_train_poly = poly_features.fit_transform(X_train)
                X_val_poly = poly_features.transform(X_val)

                # Fit polynomial regression model
                model = LinearRegression()
                model.fit(X_train_poly, y_train)

                # Predict on validation set
                y_val_pred = model.predict(X_val_poly)

                # Evaluate the model
                val_mse = mean_squared_error(y_val, y_val_pred)
                if val_mse < best_val_mse:
                    best_val_mse = val_mse
                    best_lag = num_lags
                    best_degree = degree

        print(f'Hour {hour}: Best lag: {best_lag}, Best degree: {best_degree}, Validation MSE: {best_val_mse}')
        best_lags_degrees_per_hour[hour] = (best_lag, best_degree)

        # Now train with the best lag and degree on combined train and validation sets, and test on the test set
        df_train_val = preprocess(pd.concat([df_2016, df_2017, df_2018, df_2019, df_2020, df_2021]), best_lag)
        df_test = preprocess(df_2022, best_lag)

        hour_train_val_df = df_train_val[df_train_val['Hour'] == hour]
        hour_test_df = df_test[df_test['Hour'] == hour]

        if hour_train_val_df.empty or hour_test_df.empty:
            continue

        X_train_val = hour_train_val_df[['Hour', 'DayOfWeek', 'Month', 'DayOfYear'] + [f'Lag{lag}' for lag in range(1, best_lag + 1)]]
        y_train_val = hour_train_val_df['Power']

        X_test = hour_test_df[['Hour', 'DayOfWeek', 'Month', 'DayOfYear'] + [f'Lag{lag}' for lag in range(1, best_lag + 1)]]
        y_test = hour_test_df['Power']

        # Transform features to polynomial
        poly_features = PolynomialFeatures(degree=best_degree, include_bias=False)
        X_train_val_poly = poly_features.fit_transform(X_train_val)
        X_test_poly = poly_features.transform(X_test)

        # Fit polynomial regression model
        model = LinearRegression()
        model.fit(X_train_val_poly, y_train_val)

        # Predict on test set
        y_test_pred = model.predict(X_test_poly)

        # Store predictions
        predictions_per_hour[hour] = y_test_pred

        # Evaluate the model on test set
        test_mse = mean_squared_error(y_test, y_test_pred)
        test_rmse = np.sqrt(test_mse)
        test_r2 = model.score(X_test_poly, y_test)
        test_mse_per_hour.append(test_mse)
        print(f'Hour {hour}, Test MSE with best lag {best_lag} and degree {best_degree}: {test_mse}, Test RMSE: {test_rmse}, Test R^2: {test_r2}')

    # Calculate and visualize mean MSE for each hour
    mean_test_mse = np.mean(test_mse_per_hour)
    print(f'Mean Test MSE: {mean_test_mse}')

    # Visualize the test MSE per hour
    plt.figure(figsize=(10, 6))
    plt.plot(range(24), test_mse_per_hour, marker='o', linestyle='-')
    plt.xlabel('Hour')
    plt.ylabel('Test MSE')
    plt.title('Test MSE for Each Hour')
    plt.grid(True)
    plt.show()

    return predictions_per_hour

# Example usage:
predictions = predict_next_year('juyo_2016_tohoku.csv', 'juyo_2017_tohoku.csv', 'juyo_2018_tohoku.csv', 'juyo_2019_tohoku.csv', 'juyo_2020_tohoku.csv', 'juyo_2021_tohoku.csv', 'juyo_2022_tohoku.csv')

